In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
from opendata_key import g_key
from opendata_key import open_key 
import requests
import gmaps
import time


In [41]:
open_key

'a4fc11a47e3ef30dc98115f546507e859621d12c4ca460ff1acfa357'

In [2]:
base_url = "https://public.opendatasoft.com/api/records/1.0/search"
params = {'api_key': open_key,
'dataset': 'mass-shootings-in-america',
'facet': 'city',
'facet': 'state',
'facet': 'type_of_gun_general',
'facet': 'place_type',
'facet': 'possible_motive_general',
'facet': 'history_of_mental_illness_general'
}


resp = requests.get(base_url, params = params)
resp.json()

{'error': 'Unknown dataset: mass-shootings-in-america'}

In [ ]:
city_name = []
city_name.append(resp['records']['fields']['city'][:])

In [ ]:
states = ['Texas', 'California']
city_name = []
lat = []
lng = []
state_name = []
number_killed = []
number_wounded = []
type_of_gun = []
date = []
for state in states:
# for item in range(0, len(resp['records'])):
    resp = requests.get(base_url, params = params).json()
    try:
        # city_name.append(resp['fields'][0]['city'][:])
        lat.append(resp['records']['fields'][0:]['latitude'])
        # lng.append(resp['records']['fields']['longitude'])
        # state_name.append(resp['records']['fields'][0:]['state'][:])
        # number_killed.append(resp['records']['fields']['total_number_of_fatalities'])
        # number_wounded.append(resp['records']['fields']['number_of_victims_injured'])
        # type_of_gun.append(resp['records']['fields']['type_of_gun_detailed'][:])
        # date.append(resp['records']['fields']['date'])
        # country_id.append(resp['records']['fields']['city'])
        # print(f"Processing Record {counter_record} of set {counter_set} | {city}")
    except (KeyError, IndexError):
        print("City not found. Skipping...")
print(lat)
# print(state_name)

In [ ]:
    
    city_list = []
for rec in response:
    resp = requests.get(base_url, params = params)
    response = resp.json()
    city_list.append(response['records']['fields']['city'])


In [ ]:
shooting_dic = {
    'date': '',
    'city': '',
    # 'State': '',
    # 'Latitude': '',
    # 'Longitude': '',
    # 'Number Killed': '',
    # 'Number Wounded': '',
    # 'Type of Gun': ''
}

for k, v in shooting_dic:
    resp = requests.get(base_url, params = params)
    response = resp.json()
    shooting_dic['date'].append(response['records']['fields']['date'])

In [ ]:
file_path = '../Resources/us_mass_shootings.csv'
us_mass_shooting = pd.read_csv(file_path, skiprows=7)
us_mass_shooting.head()

In [ ]:
columns = ['Date', 'City', 'State', 'Latitude', 'Longitude', 'Number killed', 'Number wounded', 'Assault weapon OR high-capacity magazine']
tx_vs_ca_df = us_mass_shooting.loc[(us_mass_shooting['State'] == 'TX')|(us_mass_shooting['State'] == 'CA'), columns]
tx_vs_ca_df.head()

In [ ]:
info_box_template = """
<dl>
<dt>Date</dt><dd>{Date}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
</dl>
"""
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(center=(30.0, 31.0), zoom_level=2)
city_info = [info_box_template.format(**row) for index, row in tx_vs_ca_df.iterrows()]
locations = tx_vs_ca_df[["Latitude", "Longitude"]]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

In [ ]:
tx_df = us_mass_shooting.loc[us_mass_shooting['State'] == 'TX']
tx_df.tail()

In [ ]:
x_axis = tx_df['Date']                      
y_axis = tx_df['Number killed']
plt.bar(x_axis, y_axis)
plt.xlabel('Years')
plt.ylabel('Number of People Killed')
plt.title('Texas Mass Shooting Last Ten Years')
plt.show()

In [ ]:
ca_df = us_mass_shooting.loc[us_mass_shooting['State'] == 'CA']
ca_df.head()

In [ ]:
x_axis = ca_df['Date']                      
y_axis = ca_df['Number killed']
plt.bar(x_axis, y_axis)
plt.xlabel('Years')
plt.ylabel('Number of People Killed')
plt.title('California Mass Shooting Last Ten Years')
plt.show()

In [ ]:
state_comp = tx_vs_ca_df.loc[:, ['State', 'Number killed', 'Number wounded']]
state_comp = state_comp.groupby('State').sum()

state_comp

In [ ]:
state_comp.plot.bar()
plt.title('Texas vs California Mass Shooting Victims Last Ten Years')
plt.xlabel('State')
plt.ylabel('Total Number of People Killed and Wounded')
plt.show()